In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(777)  # reproducibility

In [2]:
# Hyper parameters
num_epochs = 30
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [3]:
# Device configuration
device = torch.device('cuda')

# transform images to tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Write the code to define the convolutional neural network for CIFAR-10
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net(num_classes).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        # zero the parameter gradients
        optimizer.zero_grad()
        # backward + optimize
        loss.backward()
        optimizer.step()        
        
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, running_loss / 100))
            running_loss = 0.0
            

Epoch [1/30], Step [100/500], Loss: 2.0196
Epoch [1/30], Step [200/500], Loss: 1.7204
Epoch [1/30], Step [300/500], Loss: 1.6253
Epoch [1/30], Step [400/500], Loss: 1.5545
Epoch [1/30], Step [500/500], Loss: 1.5030
Epoch [2/30], Step [100/500], Loss: 1.4502
Epoch [2/30], Step [200/500], Loss: 1.4173
Epoch [2/30], Step [300/500], Loss: 1.3901
Epoch [2/30], Step [400/500], Loss: 1.3616
Epoch [2/30], Step [500/500], Loss: 1.3262
Epoch [3/30], Step [100/500], Loss: 1.3108
Epoch [3/30], Step [200/500], Loss: 1.2989
Epoch [3/30], Step [300/500], Loss: 1.2645
Epoch [3/30], Step [400/500], Loss: 1.2642
Epoch [3/30], Step [500/500], Loss: 1.2498
Epoch [4/30], Step [100/500], Loss: 1.2129
Epoch [4/30], Step [200/500], Loss: 1.2030
Epoch [4/30], Step [300/500], Loss: 1.1976
Epoch [4/30], Step [400/500], Loss: 1.2010
Epoch [4/30], Step [500/500], Loss: 1.1851
Epoch [5/30], Step [100/500], Loss: 1.1612
Epoch [5/30], Step [200/500], Loss: 1.1426
Epoch [5/30], Step [300/500], Loss: 1.1359
Epoch [5/30

In [6]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 63.37 %
